### データ読み込み

In [1]:
import pandas as pd
import numpy as np

In [2]:
test_data = pd.read_csv('02.data/test.csv')
submission_data = pd.read_csv('01.original_data/titanic/gender_submission.csv')

In [3]:
test_data.head()

,x__0:Pclass,x__1:Sex,x__2:Age,x__3:SibSp,x__4:Parch,x__5:Fare,x__6:Embarked
0,3,0,34.5,0,0,7.8292,2
1,3,1,47.0,1,0,7.0000,0
2,2,0,62.0,0,0,9.6875,2
3,3,0,27.0,0,0,8.6625,0
4,3,1,22.0,1,1,12.2875,0


In [4]:
submission_data.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


### 推論

In [5]:
import onnxruntime

In [6]:
# モデルロード
onnx_session = onnxruntime.InferenceSession('04.model/model_20210410_163453.onnx')

In [12]:
# 入力情報詳細確認
print(onnx_session.get_inputs()[0])

NodeArg(name='Input', type='tensor(float)', shape=[1, 7])


In [8]:
input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name

In [9]:
# 入力形状にあわせてリシェイプ
input_dataset = test_data.values.reshape([-1, 1, 7]).astype(np.float32)

In [10]:
# 推論
for index, input_data in enumerate(input_dataset):
    predict_result = onnx_session.run([output_name], {input_name: input_data})
    submission_data.loc[index].Survived = np.argmax(predict_result)

### CSV書き出し

In [11]:
submission_data.to_csv('05.result/submission.csv', index=False)